In [1]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search,uniform_cost_search

In [2]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError
        
    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return True



    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        
        for i in range(len(state[0])):
            if self.goal[i] != state[0][i]:
                return False 
        return True
        #if isinstance(self.goal, list):
         #   return is_in(state, self.goal)
        #else:
         #   return state[0] == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [3]:
from search import *

class Ascensor(Problem):
   
    def __init__(self, initial, goal=((3,3,3,0,2))):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        
        #operadores: subir, bajar, parar
        pos_ascensor = estado[1][0]
       
        
        #pasajeros = que pasajeros están en el ascensor
        pasajeros = estado[1][1]
      
        #plantas destino de los pasajeros
        plantas = list()
        for i in range(len(pasajeros)):
            plantas.append(self.goal[i])
       
        #lista de personas aun por recoger
        recoger = list()     
        
        for i in range(len(estado[0])):
            if i not in pasajeros and estado[0][i] != self.goal[i]:
                recoger.append(i)
       
        #lista de acciones
        accs = list()
        
        if pos_ascensor < 4:
            accs.append("Subir")
        
        if pos_ascensor > 0:
            accs.append("Bajar")
        
        #no necesitamos comprobar los pasajeros del ascensor porque se actualizan en cada nodo
        #comprobamos ademas si hay alguien que recoger o si alguien ha llegaddo a su destino
        if (pos_ascensor in recoger and len(pasajeros) < 3) or (pos_ascensor in plantas):
            accs.append("Parar")
       
        return accs
    
    def result(self,estado,accion):
        #convertimos las tuplas a listas para poder modificarlas
      
        l = list(estado)
        l[0] = list(l[0])
        l[1] = list(l[1])
        ascensor = list(estado[1]) 
        usuarios = l[0]
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Subir":
            ascensor[2] = 1                   #actualizamos el estado del ascensor
            ascensor[0] = ascensor[0] + 1     #actualizamos la planta del ascensor
            for i in range(len(ascensor[1])):             #actualizamos, para cada pasajero del ascensor, su planta
                usuarios[i] = ascensor[0] 
            
        if accion == "Bajar":
            ascensor[2] = -1                  #actualizamos el estado del ascensor
            ascensor[0] = ascensor[0] - 1     #actualizamos la planta del ascensor
            for i in range(len(ascensor[1])):               #actualizamos, para cada pasajero del ascensor, su planta
                usuarios[i] = ascensor[0] 
            
        if accion == "Parar":
            ascensor[1] = [x for x in ascensor[1] if not (self.goal[x] == ascensor[0])]  #bajamos del ascensor a los que han llegado a su planta
            #lo mismo que a continuacion
            #for e in ascensor[1]:                                                  
                #if self.goal[0][e] == ascensor[0]:
                    #del(ascensor[1][e])
            if len(ascensor[1]) < 3:                                           #subimos gente al ascensor si hay hueco
                for i in range(len(usuarios)):
                    if len(ascensor[1]) < 3 and self.goal[i] != estado[0][i]:
                        if (usuarios[i] == ascensor[0]) and (i not in ascensor[1]):
                            ascensor[1].append(i)

                   
                        
        #devolvemos las tuplas
        l[1] = ascensor
        l[0] = usuarios
        
        l[1][1] = tuple(l[1][1])
        l[0] = tuple(l[0])
        l[1] = tuple( l[1])
        
        return tuple(l)
    
    def h(self, node):
        " Return the heuristic value for a given state."
        
        return 1



In [4]:
estado_inicial = ((0,0,1,0,2), (0, (), 0))

In [5]:
class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        return 1 

In [6]:
def resuelve_ascensores(estado_inicial, algoritmo, h=None):

    p8p=Problema_con_Analizados(Ascensor(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")



In [7]:
%%time
resuelve_ascensores(estado_inicial,uniform_cost_search)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: uniform_cost_search
Longitud de la solución: 7. Nodos analizados: 51
CPU times: user 1.77 ms, sys: 84 µs, total: 1.85 ms
Wall time: 1.83 ms


In [8]:
%%time
resuelve_ascensores(estado_inicial,depth_first_graph_search)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: depth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 27
CPU times: user 612 µs, sys: 60 µs, total: 672 µs
Wall time: 619 µs


In [9]:
%%time
resuelve_ascensores(estado_inicial,depth_first_tree_search)

KeyboardInterrupt: 

In [ ]:
%%time
resuelve_ascensores(estado_inicial,breadth_first_tree_search)

In [10]:
%%time
resuelve_ascensores(estado_inicial,breadth_first_graph_search)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 7. Nodos analizados: 48
CPU times: user 1.13 ms, sys: 95 µs, total: 1.23 ms
Wall time: 1.19 ms


In [11]:
%%time
resuelve_ascensores(estado_inicial,depth_limited_search)

Solución: ['Subir', 'Subir', 'Bajar', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: depth_limited_search
Longitud de la solución: 9. Nodos analizados: 1497
CPU times: user 10.8 ms, sys: 634 µs, total: 11.4 ms
Wall time: 11 ms


In [12]:
%%time
resuelve_ascensores(estado_inicial,iterative_deepening_search)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: iterative_deepening_search
Longitud de la solución: 7. Nodos analizados: 777
CPU times: user 6.15 ms, sys: 539 µs, total: 6.68 ms
Wall time: 6.41 ms


In [13]:
#heuristicas para el problema simplificado
#primera heurística: personas que no estan en destino
def h1(node):
    state = node.state
    goals = [3,3,3,0,2]
    count = 0
    for i in range(len(state[0])):
        if (goals[i] != state[0][i]):
            count = count + 1      
    return count



#segunda heurística: similar a la distancia M
def h2(node):
    state = node.state
    goals = [3,3,3,0,2]
    #goals = node.state.goal
    count = 0
    for i in range(len(state[0])):
        count = count + abs(goals[i] - state[0][i])

    return count



#tercera heurística: distancia M + (personas fuera de destino/capacidad ascensor) * huecos ascensor
def h3(node):
    state = node.state
    goals = [3,3,3,0,2]
    no_destino = 0
    count = 0
    for i in range(len(state[0])):
        if (goals[i] != state[0][i]):
            count = abs(goals[i] - state[0][i])
            no_destino = no_destino + 1
     
    count += (no_destino/2)*(2-len(state[1][1]))

    return count




In [14]:
    for i in range(len(state[1][1])):
        if (goals[state[1][1][i]] > state[1][0]):
            arriba += 1
        elif (goals[state[1][1][i]] < state[1][0]):
            abajo += 1
        else:
            dest += 1


    if ((arriba != len(state[1][1])) or (abajo != len(state[1][1]))) and (dest != 0):
        count += 5

NameError: name 'state' is not defined

In [15]:
%%time
resuelve_ascensores(estado_inicial,astar_search,h1)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 7. Nodos analizados: 46
CPU times: user 2.03 ms, sys: 207 µs, total: 2.23 ms
Wall time: 2.14 ms


In [16]:
%%time
resuelve_ascensores(estado_inicial,astar_search,h2)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 7. Nodos analizados: 39
CPU times: user 1.67 ms, sys: 91 µs, total: 1.76 ms
Wall time: 1.74 ms


In [17]:
%%time
resuelve_ascensores(estado_inicial,astar_search,h3)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 7. Nodos analizados: 19
CPU times: user 1.63 ms, sys: 102 µs, total: 1.73 ms
Wall time: 1.71 ms


In [21]:
resuelve_ascensores(estado_inicial,best_first_graph_search,h1)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: best_first_graph_search
Heurística: h1
Longitud de la solución: 7. Nodos analizados: 56


In [22]:
resuelve_ascensores(estado_inicial,best_first_graph_search,h2)

Solución: ['Subir', 'Parar', 'Subir', 'Bajar', 'Parar', 'Subir', 'Bajar', 'Parar', 'Subir', 'Subir']
Algoritmo: best_first_graph_search
Heurística: h2
Longitud de la solución: 10. Nodos analizados: 47


In [23]:
resuelve_ascensores(estado_inicial,best_first_graph_search,h3)

Solución: ['Subir', 'Parar', 'Bajar', 'Parar', 'Subir', 'Subir', 'Subir']
Algoritmo: best_first_graph_search
Heurística: h3
Longitud de la solución: 7. Nodos analizados: 25
